In [1]:
#importing the required libraries
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv


In [2]:
#reading the dataset
dfSub = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv')
dfStarter = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv')
dfTrain = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')
dfTest = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv')

In [3]:
#Making a Copy of The Dataset
#I would be working with this copy through out the project while the original dataframe would be for referencing when needed .
df_Sub = dfSub.copy()
df_Starter = dfStarter.copy()
df_Train = dfTrain.copy()
df_Test =dfTest.copy()

In [4]:
print(df_Train.shape)
print(df_Test.shape)
print(df_Sub.shape)
print(df_Starter.shape)

(122265, 7)
(25080, 3)
(25080, 2)
(3142, 26)


Along with the row id, cfips, county, and state, first day of month, microbusiness density, and active fields are included in train data. Only row id, cfips, and first day of month are present in the test data. Microbusiness density for a specific cfips and at a specific first day of month is the value to be predicted.

In [5]:
#information on the test dataset
df_Test.head(3)

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01


In [6]:
df_Test.first_day_of_month.unique()

array(['2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01',
       '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01'],
      dtype=object)

Test dataset spans from, November 2022 through June 2023.

In [7]:
#information on train datset
df_Train.head(2)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198


In [8]:
df_Train.first_day_of_month.unique()

array(['2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01',
       '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01',
       '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01',
       '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01',
       '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01',
       '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
       '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01',
       '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01',
       '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01',
       '2022-08-01', '2022-09-01', '2022-10-01'], dtype=object)

Train dataset spans from August 2019 through October 2022.

In [9]:
df_Train['state'].value_counts()

Texas                   9906
Georgia                 6201
Virginia                5070
Kentucky                4680
Missouri                4485
Kansas                  4095
Illinois                3978
North Carolina          3900
Iowa                    3861
Tennessee               3705
Nebraska                3588
Indiana                 3588
Ohio                    3432
Minnesota               3393
Michigan                3237
Mississippi             3198
Oklahoma                3003
Arkansas                2925
Wisconsin               2808
Pennsylvania            2613
Alabama                 2613
Florida                 2613
South Dakota            2535
Colorado                2496
Louisiana               2496
New York                2418
California              2262
Montana                 2184
West Virginia           2145
North Dakota            2067
South Carolina          1794
Idaho                   1716
Washington              1521
Oregon                  1404
New Mexico    

In [10]:
df_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [11]:
#determing the number of rows of train and test dataset
len(df_Test), len(df_Train)

(25080, 122265)

## Data Cleaning

In [12]:
#function to seperate the first day of the month from the cfips
def get_first_day_of_month(item):
    item = item.split('_')[1]
    return item

def get_cfips(item):
    item = item.split('_')[0]
    return item

In [13]:
#applying the defined function
df_Sub['first_day_of_month'] = df_Sub['row_id'].apply(get_first_day_of_month)
df_Sub['cfips'] = df_Sub['row_id'].apply(get_cfips)

In [14]:
#converting the date to datetime
df_Test['first_day_of_month'] = pd.to_datetime(df_Test['first_day_of_month'])
df_Train['first_day_of_month'] = pd.to_datetime(df_Train['first_day_of_month'])
df_Sub['first_day_of_month'] = pd.to_datetime(df_Sub['first_day_of_month'])

In [15]:
#dropping the `row_id` column
df_Test.drop(['row_id'],axis = 1, inplace = True)
df_Train.drop(['row_id'],axis = 1, inplace = True)
df_Sub.drop(['row_id'],axis = 1, inplace = True)

In [16]:
#merging the dataset 
df_trainMerged = df_Train.merge(df_Starter, on = 'cfips', how = 'left')
df_trainMerged.head(2)

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0


In [17]:
#after merging, we can see that some columns now have null values which needs to be cleaned up
df_trainMerged.isna().sum()

cfips                     0
county                    0
state                     0
first_day_of_month        0
microbusiness_density     0
active                    0
pct_bb_2017               0
pct_bb_2018               0
pct_bb_2019               0
pct_bb_2020               0
pct_bb_2021               0
pct_college_2017          0
pct_college_2018          0
pct_college_2019          0
pct_college_2020          0
pct_college_2021          0
pct_foreign_born_2017     0
pct_foreign_born_2018     0
pct_foreign_born_2019     0
pct_foreign_born_2020     0
pct_foreign_born_2021     0
pct_it_workers_2017       0
pct_it_workers_2018      39
pct_it_workers_2019       0
pct_it_workers_2020       0
pct_it_workers_2021       0
median_hh_inc_2017        0
median_hh_inc_2018       39
median_hh_inc_2019        0
median_hh_inc_2020       39
median_hh_inc_2021       39
dtype: int64

In [18]:
#sample is the df_trainMerged['pct_bb_2018']column which we can also view below
df_null = df_trainMerged[df_trainMerged['pct_bb_2018'].isnull()]
df_null

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021


In [19]:
df_trainMerged['median_hh_inc_2018']= df_trainMerged['median_hh_inc_2018'].fillna(df_trainMerged['median_hh_inc_2018'].median())

## Exploratory Data Analysis

In [20]:
a = df_trainMerged['median_hh_inc_2018'].value_counts()
a.head(50)

48667.0    117
40978.0    117
35509.0    117
49891.0    117
42604.0    117
37188.0    117
44313.0    117
47500.0    117
40000.0    117
42689.0    117
61145.0     78
48563.0     78
56507.0     78
49233.0     78
44350.0     78
48853.0     78
63902.0     78
35931.0     78
40837.0     78
57899.0     78
50905.0     78
48125.0     78
43355.0     78
76912.0     78
46182.0     78
47518.0     78
45426.0     78
54184.0     78
42500.0     78
56071.0     78
50511.0     78
40810.0     78
43893.0     78
55832.0     78
54249.0     78
40769.0     78
51250.0     78
52794.0     78
39002.0     78
45000.0     78
49146.0     78
52620.0     78
51920.0     78
60466.0     78
47497.0     78
51056.0     78
54197.0     78
53021.0     78
36875.0     78
67591.0     78
Name: median_hh_inc_2018, dtype: int64

In [21]:
y = df_trainMerged.loc[ df_trainMerged['median_hh_inc_2018'].isnull()]
y

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
